In [1]:
import pandas as pd
from textblob import TextBlob
from gensim import corpora, models
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string

C:\Users\Saurabh Wani\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
cd E:/Projects/Sentiment Analysis/Datasets/Amazon Alexa

E:\Projects\Sentiment Analysis\Datasets\Amazon Alexa


In [3]:
dataset = pd.read_csv("Reviews.csv")

In [4]:
dataset.head(2)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...


In [5]:
New_data = dataset[{'reviews.rating', 'reviews.text'}]
New_data.head()

,reviews.text,reviews.rating
0,I thought it would be as big as small paper bu...,3
1,This kindle is light and easy to use especiall...,5
2,Didnt know how much i'd use a kindle so went f...,4
3,I am 100 happy with my purchase. I caught it o...,5
4,Solid entry level Kindle. Great for kids. Gift...,5


In [6]:
New_data.count()

reviews.text      5000
reviews.rating    5000
dtype: int64

In [7]:
clean_data = New_data.dropna()
clean_data.head()

,reviews.text,reviews.rating
0,I thought it would be as big as small paper bu...,3
1,This kindle is light and easy to use especiall...,5
2,Didnt know how much i'd use a kindle so went f...,4
3,I am 100 happy with my purchase. I caught it o...,5
4,Solid entry level Kindle. Great for kids. Gift...,5


In [8]:
clean_data.count()

reviews.text      5000
reviews.rating    5000
dtype: int64

Preprocessong

In [9]:
from nltk import sent_tokenize
rd = []
for r in clean_data['reviews.text']:
    sentences = sent_tokenize(r)
    #print(sentences[0])
    tokens = word_tokenize(r)
    #print(tokens[:100])
    words = [word for word in tokens if word.isalpha()]
    #print(words[:100])
    stop_words = stopwords.words('english')
    #print(stop_words)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    #print(words[:100])
    rd.append(words)

In [10]:
joined_list = []
for i in range(len(clean_data['reviews.text'])):
    s = ' '
    t = s.join(rd[i])
    joined_list.append(t)

In [26]:
pol = np.zeros(len(clean_data['reviews.text']))
sub = np.zeros(len(clean_data['reviews.text'])) 

for i in range(len(clean_data['reviews.text'])):
    r = clean_data['reviews.text'][i]
    analysis = TextBlob(r)
    Polarity = float(format(analysis.sentiment.polarity))
    Subjectivity = float(format(analysis.sentiment.subjectivity))
    pol[i] = Polarity
    sub[i] = Subjectivity
    

In [27]:
rat = np.zeros(len(clean_data['reviews.text']))
for i in range(len(clean_data['reviews.text'])):
    rating = clean_data['reviews.rating'][i]
    rat[i] = rating

In [28]:
sentiment = pd.DataFrame(
    {'Polarity': pol,
     'Subjectivity': sub,
     'Ratings': rat
    })

In [29]:
sentiment.head()

,Polarity,Subjectivity,Ratings
0,-0.108974,0.348718,3.0
1,0.277778,0.844444,5.0
2,0.165625,0.525000,4.0
3,0.240497,0.544528,5.0
4,0.468750,0.568750,5.0


In [30]:
filter_3 = ((sentiment['Ratings'] > 3) & (sentiment['Polarity'] < 0))
filter_3.head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [42]:
m = sentiment.loc[((sentiment['Ratings'] > 3) & (sentiment['Polarity'] < 0)) | ((sentiment['Ratings'] < 3) & (sentiment['Polarity'] > 0))]
m.head()

,Polarity,Subjectivity,Ratings
7,-0.200000,0.400000,4.0
9,-0.100000,0.600000,5.0
20,0.233333,0.255556,1.0
49,-0.025000,0.233333,4.0
70,0.338542,0.570833,1.0


In [32]:
drop_list = list(m.index.values)

In [43]:
final_data = sentiment.drop(drop_list)
final_data.describe()

,Polarity,Subjectivity,Ratings
count,4794.000000,4794.000000,4794.000000
mean,0.384399,0.561845,4.632457
std,0.243610,0.214519,0.661351
min,-0.750000,0.000000,1.000000
25%,0.216667,0.458333,4.000000
50%,0.377778,0.577778,5.000000
75%,0.533333,0.700000,5.000000
max,1.000000,1.000000,5.000000


After we have final dataset, We're ready to do some predictive analysis

In [34]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB


In [45]:
knn = KNeighborsClassifier(n_neighbors=83)

features = final_data[{'Polarity','Subjectivity'}]
label = final_data['Ratings']
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.05, random_state=100)
knn.fit(x_train, y_train)

pred_knn = knn.predict(x_test)
acc_knn = accuracy_score(y_true = y_test, y_pred = pred_knn)*100

print("Accuracy of KNN classifier: " + str(acc_knn) + " %")

Accuracy of KNN classifier: 73.75 %


In [46]:
RF = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 0)

features = final_data[{'Polarity','Subjectivity'}]
label = final_data['Ratings']
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.05, random_state=100)
RF.fit(x_train, y_train)

pred_RF = RF.predict(x_test)
acc_RF = accuracy_score(y_true = y_test, y_pred = pred_RF)*100

print("Accuracy of Random Forest classifier: " + str(acc_RF) + " %")

Accuracy of Random Forest classifier: 72.5 %


In [47]:
features = final_data[{'Polarity','Subjectivity'}]
label = final_data['Ratings']

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.15, random_state=42)

LearningRates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for lr in LearningRates:
    GB = GradientBoostingClassifier(n_estimators=15, learning_rate = lr, max_features=2, max_depth = 5, random_state = 0)
    GB.fit(x_train, y_train)
    pred_GB = GB.predict(x_test)
    acc_GB = accuracy_score(y_true = y_test, y_pred = pred_GB)*100
    print('Learning rate: '+str(lr))
    print("Accuracy of Gradient Boosting classifier: " + str(acc_GB) + " %")


Learning rate: 0.05
Accuracy of Gradient Boosting classifier: 73.47222222222223 %
Learning rate: 0.1
Accuracy of Gradient Boosting classifier: 73.05555555555556 %
Learning rate: 0.25
Accuracy of Gradient Boosting classifier: 71.94444444444444 %
Learning rate: 0.5
Accuracy of Gradient Boosting classifier: 71.11111111111111 %
Learning rate: 0.75
Accuracy of Gradient Boosting classifier: 71.25 %
Learning rate: 1
Accuracy of Gradient Boosting classifier: 70.97222222222223 %


In [48]:
GB = GradientBoostingClassifier(n_estimators=15, learning_rate = 0.05, max_features=2, max_depth = 5, random_state = 0)
GB.fit(x_train, y_train)
pred_GB = GB.predict(x_test)
acc_GB = accuracy_score(y_true = y_test, y_pred = pred_GB)*100
print("Accuracy of Gradient Boosting classifier: " + str(acc_GB) + " %")

Accuracy of Gradient Boosting classifier: 73.47222222222223 %


In [49]:
gnb = GaussianNB(priors=None, var_smoothing=1e-09)
features = final_data[{'Polarity','Subjectivity'}]
label = final_data['Ratings']

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.15, random_state=0)
gnb.fit(x_train, y_train)

pred_gnb = gnb.predict(x_test)
acc_gnb = accuracy_score(y_true = y_test, y_pred = pred_gnb)*100

print("Accuracy of Naive Bayes classifier: " + str(acc_gnb) + " %")
print(gnb.class_prior_)

Accuracy of Naive Bayes classifier: 71.94444444444444 %
[0.00810015 0.00466372 0.04099165 0.24153166 0.70471281]


In [50]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, pred_GB, average=None)

(array([0.        , 0.        , 0.33333333, 0.25      , 0.72063037]),
 array([0.        , 0.        , 0.03333333, 0.0060241 , 0.9748062 ]),
 array([0.        , 0.        , 0.06060606, 0.01176471, 0.82866557]),
 array([  5,   3,  30, 166, 516], dtype=int64))

In [51]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_GB)

array([[  0,   0,   0,   0,   5],
       [  0,   0,   0,   0,   3],
       [  2,   0,   1,   0,  27],
       [  2,   3,   0,   1, 160],
       [  3,   5,   2,   3, 503]], dtype=int64)

Precision and recall are not as good

Try Smote function or over/under sampling

In [53]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)

x = final_data[{'Polarity','Subjectivity'}]
y = final_data['Ratings']
X_resampled, y_resampled = ros.fit_resample(x,y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))
print(sorted(Counter(y).items()))

[(1.0, 3387), (2.0, 3387), (3.0, 3387), (4.0, 3387), (5.0, 3387)]
[(1.0, 38), (2.0, 22), (3.0, 197), (4.0, 1150), (5.0, 3387)]


In [54]:
knn = KNeighborsClassifier(n_neighbors=3)

features = final_data[{'Polarity','Subjectivity'}]
label = final_data['Ratings']
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=100)
knn.fit(x_train, y_train)

pred_knn = knn.predict(x_test)
acc_knn = accuracy_score(y_true = y_test, y_pred = pred_knn)*100

print("Accuracy of KNN classifier: " + str(acc_knn) + " %")

Accuracy of KNN classifier: 79.97165800661314 %


In [55]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_knn)

array([[766,   0,   0,   0,  54],
       [  0, 739,   0,   0, 100],
       [  0,   0, 774,  19,  52],
       [  1,   0, 103, 602, 165],
       [  0,   1, 104, 249, 505]], dtype=int64)

In [56]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, pred_knn, average=None)

(array([0.99869622, 0.99864865, 0.78899083, 0.69195402, 0.57648402]),
 array([0.93414634, 0.88081049, 0.91597633, 0.69115959, 0.5878929 ]),
 array([0.96534342, 0.93603547, 0.84775465, 0.69155658, 0.58213256]),
 array([820, 839, 845, 871, 859], dtype=int64))

In [57]:
RF = RandomForestClassifier(n_estimators = 70, criterion = 'entropy', random_state = 0)

features = final_data[{'Polarity','Subjectivity'}]
label = final_data['Ratings']

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=100)

RF.fit(x_train, y_train)

pred_RF = RF.predict(x_test)

acc_RF = accuracy_score(y_true = y_test, y_pred = pred_RF)*100

print("Accuracy of Random Forest classifier: " + str(acc_RF) + " %")

Accuracy of Random Forest classifier: 85.23854511100613 %


In [58]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_RF)

array([[766,  54,   0,   0,   0],
       [  0, 839,   0,   0,   0],
       [  0,  44, 775,  14,  12],
       [  0,  24,  62, 678, 107],
       [  0,  46,  49, 213, 551]], dtype=int64)

In [59]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, pred_RF, average=None)

(array([1.        , 0.83316783, 0.87471783, 0.74917127, 0.82238806]),
 array([0.93414634, 1.        , 0.91715976, 0.77841561, 0.64144354]),
 array([0.96595208, 0.90899242, 0.89543616, 0.76351351, 0.7207325 ]),
 array([820, 839, 845, 871, 859], dtype=int64))